## **Imports & Setup**

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
from nltk.stem import WordNetLemmatizer
import builtins
import numpy as np
from nltk.tokenize import word_tokenize

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
nltk.download('wordnet')



In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
bucket_name = '209234103_final'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "multistream" in b.name:
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

## **Set Data**

In [ ]:
Body = parquetFile.select("text", "id").rdd
Title = parquetFile.select("title", "id").rdd
Pages_Links = parquetFile.select("id","anchor_text").rdd

## **tokenize**

In [ ]:
# Function to get the HTML pattern
def get_html_pattern():
    return r"<[^>]+>"
# Function to get the date pattern
def get_date_pattern():
    return r"(?:Jan(?:uary)?|Mar(?:ch)?|May|Jul(?:y)?|Aug(?:ust)?|Oct(?:ober)|Dec(?:ember)?)\s(?:0[1-9]|1[0-9]|2[0-9]|3[01]|[1-9]),\s\d{4}|(?:Apr(?:il)?|Jun(?:e)?|Sep(?:tember)?|Nov(?:ember)?)\s(?:0[1-9]|1[0-9]|2[0-9]|3[0]|[1-9]),\s\d{4}|(?:Feb(?:ruary)?)\s(?:0[1-9]|1[0-9]|2[0-9]|[1-9]),\s\d{4}|(?:0[1-9]|1[0-9]|2[0-9]|3[01]|[1-9])\s(?:Jan(?:uary)?|Mar(?:ch)?|May|Jul(?:y)?|Aug(?:ust)?|Oct(?:ober)|Dec(?:ember)?),\s\d{4}|(?:0[1-9]|1[0-9]|2[0-9]|3[01]|[1-9])\s(?:Jan(?:uary)?|Mar(?:ch)?|May|Jul(?:y)?|Aug(?:ust)?|Oct(?:ober)|Dec(?:ember)?)\s\d{4}|(?:0[1-9]|1[0-9]|2[0-9]|3[0]|[1-9])\s(?:Apr(?:il)?|Jun(?:e)?|Sep(?:tember)?|Nov(?:ember)?)\s\d{4}|(?:0[1-9]|1[0-9]|2[0-9]|[1-9])\s(?:Feb(?:ruary)?)\s\d{4}"

# Function to get the time pattern
def get_time_pattern():
    return r"((?:[0-1]\d[0-5]\d|[2][0-4][0-5]\d|[0-1]\d\.[0-5]\d|[2][0-4]\.[0-5]\d)(?:[APM]{2}|[apm.]{4}))|(\b(?:[0-1]\d|[2][0-4]|\d{1})(?::[0-5]\d){2}\b)"

# Function to get the percent pattern
def get_percent_pattern():
    return r"\d+(\.\d+)?%"

# Function to get the number pattern
def get_number_pattern():
    return r"(?:(?<=[\s\(])|^)[+-]?(?:(?:\d+(?:,\d{3})*(?:\.\d+)?)\b|\(\d+\))(?=[\s,:).]|$)"

# Function to get the word pattern
def get_word_pattern():
    return r"\b(?<![-,:=\+\$\w])\d?[A-Za-z\'-]+"

RE_TOKENIZE = re.compile(rf"""
(
    # parsing html tags
     (?P<HTMLTAG>{get_html_pattern()})
    # dates
    |(?P<DATE>{get_date_pattern()})
    # time
    |(?P<TIME>{get_time_pattern()})
    # Percents
    |(?P<PERCENT>{get_percent_pattern()})
    # Numbers
    |(?P<NUMBER>{get_number_pattern()})
    # Words
    |(?P<WORD>{get_word_pattern()})
    # space
    |(?P<SPACE>[\s\t\n]+)
    # everything else
    |(?P<OTHER>.))""",  re.MULTILINE | re.IGNORECASE | re.VERBOSE | re.UNICODE)

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w)""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

In [ ]:
def tokenize(text):

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = ' '.join(tokens)
    tokens = [v for match in RE_TOKENIZE.finditer(tokens)
                 for k, v in match.groupdict().items()
                 if v is not None and k in ['WORD','NUMBER']]

    tokens = [token for token in tokens if token not in all_stopwords]

    return tokens

In [ ]:
def tokenize_stemmer(text):

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = ' '.join(tokens)
    tokens = [v for match in RE_TOKENIZE.finditer(tokens)
                 for k, v in match.groupdict().items()
                 if v is not None and k in ['WORD','NUMBER']]

    tokens = [token for token in tokens if token not in all_stopwords]

    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

In [ ]:
def tokenize_lemmatization(text):
    
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = ' '.join(tokens)
    tokens = [v for match in RE_TOKENIZE.finditer(tokens)
                 for k, v in match.groupdict().items()
                 if v is not None and k in ['WORD','NUMBER']]

    tokens = [token for token in tokens if token not in all_stopwords]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

## **functions**

### word count:

In [ ]:
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''

    tokens = tokenize(text)

    # Calculate term frequency (tf)
    tf_counts = Counter(tokens)

    # Create a list of tuples in the form (token, (doc_id, tf))
    result = [(token, (id, tf)) for token, tf in tf_counts.items()]

    # Return the list of token count tuples
    return result

In [ ]:
def word_count_stemmer(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''

    tokens = tokenize_stemmer(text)

    # Calculate term frequency (tf)
    tf_counts = Counter(tokens)

    # Create a list of tuples in the form (token, (doc_id, tf))
    result = [(token, (id, tf)) for token, tf in tf_counts.items()]

    # Return the list of token count tuples
    return result

In [ ]:
def word_count_lemmatization(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''

    tokens = tokenize_lemmatization(text)

    # Calculate term frequency (tf)
    tf_counts = Counter(tokens)

    # Create a list of tuples in the form (token, (doc_id, tf))
    result = [(token, (id, tf)) for token, tf in tf_counts.items()]

    # Return the list of token count tuples
    return result

### reduce_word_counts

In [ ]:
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
    # Sort the list of tuples by the first element (wiki_id)
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])

    return sorted_pl

### calculate df:

In [ ]:
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
    # Count the number of documents for each token
    df_rdd = postings.mapValues(lambda x : len(x))

    return df_rdd

### Partition postings and write:

In [ ]:
def partition_postings_and_write(postings, base_dir, bucket_name):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''

    # Map each posting to its bucket
    postings = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

    # Group postings by bucket and write posting lists to disk
    posting_locations = postings.map(lambda x : InvertedIndex.write_a_posting_list(x, base_dir, bucket_name))
    return posting_locations

### calc_tf_idf

In [ ]:
def calc_tf_idf(tf, df, N, doc_len):
    tf_idf = (tf / doc_len) * math.log(N / df,2)
    return tf_idf

## **Inverted Index Title**

In [ ]:
def Inverted_Index_Title(title, base_dir, bucket_name, folder, index):
    # word counts map
    word_counts = title.flatMap(lambda x: word_count(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)

    # filtering postings and calculate df
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()
    
    # partition posting lists and write out
    _ = partition_postings_and_write(postings, base_dir, bucket_name).collect()

    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=base_dir):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

    # Create inverted index instance
    title_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    title_index.posting_locs = super_posting_locs

    # Term Frequency Calculation for Document
    tf_doc = postings.flatMap(lambda term: [(doc_id, (term[0], frequency)) for doc_id, frequency in term[1]]).groupByKey().mapValues(list)
    
    # Document Length Calculation
    doc_len = tf_doc.map(lambda doc_id: (doc_id[0], np.sum(frequency for _, frequency in doc_id[1]))).collectAsMap()

    # Add the token - df dictionary to the inverted index
    title_index.df = w2df_dict
    title_index.doc_len = doc_len
    title_index.N= title.count()

    tf_idf = tf_doc.flatMap(lambda doc_id: [(doc_id[0],calc_tf_idf(frequency, title_index.df[term], title_index.N , title_index.doc_len[doc_id[0]])) for term, frequency in doc_id[1]]).groupByKey()
    tf_idf_res = tf_idf.map(lambda doc_id: (doc_id[0],np.sum(math.pow(tf_idf,2) for tf_idf in doc_id[1]))).collectAsMap()
    idf = {term: math.log(title_index.N / tf, 2) for term, tf in w2df_dict.items()}
    
    title_index.weights_square = tf_idf_res 
    title_index.idf = idf

    # write the global stats out
    title_index.write_index('.', index)

    # upload to gs
    index_src = f'{index}.pkl'
    index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
    !gsutil cp $index_src $index_dst
    !gsutil ls -lh $index_dst


In [ ]:
Inverted_Index_Title(Title,"Title_no_stem",bucket_name,"Title_no_stem", "Title_no_stem_index")

## **Inverted Index Title Stemmer**

In [ ]:
def Inverted_Index_Title_stemmer(title, base_dir, bucket_name, folder, index):
    # word counts map
    word_counts = title.flatMap(lambda x: word_count_stemmer(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)

    # filtering postings and calculate df
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()
    
    # partition posting lists and write out
    _ = partition_postings_and_write(postings, base_dir, bucket_name).collect()

    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=base_dir):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

    # Create inverted index instance
    title_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    title_index.posting_locs = super_posting_locs

    # Term Frequency Calculation for Document
    tf_doc = postings.flatMap(lambda term: [(doc_id, (term[0], frequency)) for doc_id, frequency in term[1]]).groupByKey().mapValues(list)
    
    # Document Length Calculation
    doc_len = tf_doc.map(lambda doc_id: (doc_id[0], np.sum(frequency for _, frequency in doc_id[1]))).collectAsMap()

    # Add the token - df dictionary to the inverted index
    title_index.df = w2df_dict
    title_index.doc_len = doc_len
    title_index.N= title.count()

    tf_idf = tf_doc.flatMap(lambda doc_id: [(doc_id[0],calc_tf_idf(frequency, title_index.df[term], title_index.N , title_index.doc_len[doc_id[0]])) for term, frequency in doc_id[1]]).groupByKey()
    tf_idf_res = tf_idf.map(lambda doc_id: (doc_id[0],np.sum(math.pow(tf_idf,2) for tf_idf in doc_id[1]))).collectAsMap()
    idf = {term: math.log(title_index.N / tf, 2) for term, tf in w2df_dict.items()}
    
    title_index.weights_square = tf_idf_res 
    title_index.idf = idf

    # write the global stats out
    title_index.write_index('.', index)

    # upload to gs
    index_src = f'{index}.pkl'
    index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
    !gsutil cp $index_src $index_dst
    !gsutil ls -lh $index_dst


In [ ]:
Inverted_Index_Title_stemmer(Title,"Title",bucket_name,"Title", "Title_index")

## **Inverted Index Title lemmatization**

In [ ]:
def Inverted_Index_Title_lemmatization(title, base_dir, bucket_name, folder, index):
    # word counts map
    word_counts = title.flatMap(lambda x: word_count_lemmatization(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)

    # filtering postings and calculate df
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()
    
    # partition posting lists and write out
    _ = partition_postings_and_write(postings, base_dir, bucket_name).collect()

    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=base_dir):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

    # Create inverted index instance
    title_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    title_index.posting_locs = super_posting_locs

    # Term Frequency Calculation for Document
    tf_doc = postings.flatMap(lambda term: [(doc_id, (term[0], frequency)) for doc_id, frequency in term[1]]).groupByKey().mapValues(list)
    
    # Document Length Calculation
    doc_len = tf_doc.map(lambda doc_id: (doc_id[0], np.sum(frequency for _, frequency in doc_id[1]))).collectAsMap()

    # Add the token - df dictionary to the inverted index
    title_index.df = w2df_dict
    title_index.doc_len = doc_len
    title_index.N= title.count()

    tf_idf = tf_doc.flatMap(lambda doc_id: [(doc_id[0],calc_tf_idf(frequency, title_index.df[term], title_index.N , title_index.doc_len[doc_id[0]])) for term, frequency in doc_id[1]]).groupByKey()
    tf_idf_res = tf_idf.map(lambda doc_id: (doc_id[0],np.sum(math.pow(tf_idf,2) for tf_idf in doc_id[1]))).collectAsMap()
    idf = {term: math.log(title_index.N / tf, 2) for term, tf in w2df_dict.items()}
    
    title_index.weights_square = tf_idf_res 
    title_index.idf = idf

    # write the global stats out
    title_index.write_index('.', index)

    # upload to gs
    index_src = f'{index}.pkl'
    index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
    !gsutil cp $index_src $index_dst
    !gsutil ls -lh $index_dst


In [ ]:
Inverted_Index_Title_lemmatization(Title,"Title_lemm",bucket_name,"Title_lemm", "Title_lemm_index")